<a href="https://colab.research.google.com/github/Nikhil-Nandam/TensorFlow-Notebooks/blob/main/08_Introduction_to_NLP_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences, text or speech).

Another common term for NLP Problems is sequence to sequence problems (seq2seq).

In [1]:
!nvidia-smi

Thu Jun  9 04:29:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import a series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-06-09 04:29:03--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-06-09 04:29:03 (115 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

See the [original source](https://www.kaggle.com/competitions/nlp-getting-started).

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-06-09 04:29:06--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 74.125.69.128, 173.194.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-06-09 04:29:06 (158 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would to us Python.

But it is better to visualize it right away.

So, another way to do this would be through pandas...

In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
# Shuffle training dataset
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)   # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index + 5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not real disaster)
Text:
Apollo Brown - Detonate (feat. M.O.P.) by Mello Music Group via #soundcloud https://t.co/PRojeAvG8T

---

Target: 0 (not real disaster)
Text:
Can you recommend anyone for this #job? RN Emergency Services Full Time 3p - 3\:30a Rose de Lima Campus - http://t.co/xQrLEWiA4x #Hiring

---

Target: 1 (real disaster)
Text:
Australia's Ashes disaster - how the collapse unfolded at Trent Bridge - Telegraph http://t.co/6FYnerMUsG

---

Target: 0 (not real disaster)
Text:
@ArgentaElite haha traumatised !!!! Hell no I want a job ?? xxx

---

Target: 1 (real disaster)
Text:
@CAgov If 90BLKs&amp;8WHTs colluded 2 take WHT F @USAgov AUTH Hostage&amp;2 make her look BLK w/Bioterrorism&amp;use her lgl/org IDis ID still hers?@VP

---



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split the training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(), 
                                                                            train_df_shuffled["target"].to_numpy(), 
                                                                            random_state=42, 
                                                                            test_size=0.1)    # use 10% of training data for validation split

In [13]:
# Check the lengths 
len(train_sentences), len(train_labels), len(val_sentences), len (val_labels)

(6851, 6851, 762, 762)

In [14]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text Vectorization (Tokenization)

In [15]:
from pandas.core.common import standardize_mapping
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None,    # how many words in the vocabulary (automatically add <OOV> (Out of Vocabulary))
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,        # create groups of n-words
                                    output_mode="int",
                                    output_sequence_length=None,    # how long do you want your sequences to be
                                    pad_to_max_tokens=False)

In [16]:
len(train_sentences[0].split())

7

In [17]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [18]:
# Setup text vectorization variables
max_vocab_length = 10000    # max number of words to have in our vocabulary
max_length = 15             # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [19]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [20]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 I added a video to a @YouTube playlist http://t.co/1vjAlJA1SX GTA 5 Funny Moments - 'OBLITERATION!' (GTA 5 Online Funny Moments)        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   8,  911,    3,   72,    5,    3,  114,  998,    1, 2099,  180,
        1136, 1809,  536, 2099]])>

In [22]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()   # get all of the unique words in the training data
top_5_words = words_in_vocab[:5]        # get the most common words
bottom_5_words = words_in_vocab[-5:]    # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use [TensorFlow's Embedding layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)

The parameters we care the most about for our embedding layer:
* `input_dim`: the size of our vocabulary
* `output_dim`: the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length`: length of the sequences being passed to the embedding layer

In [23]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim=max_vocab_length,   # set the input shape
                      output_dim=128,               # output shape
                      embeddings_initializer="uniform",
                      input_length=max_length)      # how long is each input

embedding        

In [24]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Last time I checked Lots of injuries over the course of time = injury prone. 

@Calum36Chambers am I wrong?        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02719653, -0.02409813,  0.01559227, ...,  0.04076797,
         -0.01633162, -0.0164988 ],
        [-0.03849864, -0.02063037, -0.03804022, ..., -0.00878435,
          0.04197775, -0.02450647],
        [ 0.04641682, -0.00937628,  0.03289366, ..., -0.01132872,
          0.01944743,  0.00831433],
        ...,
        [ 0.03896446,  0.01721321, -0.02084498, ..., -0.04515803,
         -0.03538098, -0.0453921 ],
        [-0.03259635, -0.04856355, -0.00916421, ..., -0.04132992,
         -0.0298722 ,  0.0121708 ],
        [ 0.00699065, -0.00821134, -0.01300017, ..., -0.01938957,
         -0.01056659, -0.01502104]]], dtype=float32)>

In [25]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 2.71965303e-02, -2.40981337e-02,  1.55922659e-02,  3.17853801e-02,
        -3.61665338e-03, -4.73795906e-02,  2.46576183e-02, -1.83449872e-02,
        -2.89471503e-02,  4.92786802e-02, -4.83745337e-02, -3.46679464e-02,
        -4.34863567e-03,  9.58936289e-03, -4.74725738e-02,  3.29396240e-02,
         1.96189024e-02,  2.39438750e-02,  6.25624508e-03,  1.44951679e-02,
         8.27265903e-03, -4.12592664e-02,  1.44839324e-02, -2.44781133e-02,
         3.00755389e-02,  7.66208023e-03,  7.13746622e-03, -3.87198813e-02,
        -8.14555958e-03, -3.82383093e-02, -3.61626633e-02, -4.01579365e-02,
         1.04492083e-02, -7.47497007e-03,  8.91347975e-03, -1.70768276e-02,
         9.50060040e-03, -3.32439095e-02,  3.28320973e-02,  1.20189562e-02,
         2.21085809e-02, -1.74855962e-02,  4.34402935e-02, -3.87206674e-05,
         3.78825516e-03, -4.01683338e-02, -1.40220746e-02,  2.08277591e-02,
        -3.95121425e-03, -3.08535248e-0

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.
* Model 0: Naïve Bayes (baseline), this is from [Sklearn ML map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with TensorFlow
* Create a model
* Build the model
* Fit the model
* Evaluate the model

### Model 0: Getting a baseline

As with all Machine Learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll be using Sklearn's Multinomial Naïïve Bayes using the TF_IDF formula to convert our words to numbers.

> 🔑**Note:** It's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),   # convert words to numbers using tfidf
    ("clf", MultinomialNB()),       # model the text
])

# Fit the pipeline to training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score * 100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [28]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however this will be cumbersome and could easily be fixed with a function.

Let's create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-Score

For more detail on various evaluation metrics refer [Sklearn's Documentation](https://scikit-learn.org/stable/modules/model_evaluation.html)

In [29]:
# Funtion to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1-score of a binary classification model.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100

    # Calculate model precision, recall and f1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1_score": model_f1}

    return model_results

In [30]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1_score': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A Simple Dense Model

In [31]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [32]:
# Build model with the Functional API
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling1D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

inputs = Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text to numbers
x = embedding(x)    # create an embedding of the numberized inputs
x = GlobalAveragePooling1D(name="global_avg_pool_layer")(x) # condense the feature vector for each token to one vector
outputs = Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation function
model_1 = Model(inputs, outputs, name="model_1_dense")

In [33]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [34]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])

In [35]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20220609-042915
Epoch 1/5
215/215 [==============================] - 9s 11ms/step - loss: 0.6143 - accuracy: 0.6872 - val_loss: 0.5357 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.4425 - accuracy: 0.8180 - val_loss: 0.4709 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3477 - accuracy: 0.8610 - val_loss: 0.4622 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2850 - accuracy: 0.8899 - val_loss: 0.4615 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2385 - accuracy: 0.9099 - val_loss: 0.4784 - val_accuracy: 0.7848


In [36]:
# Check the results 
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4784 - accuracy: 0.7848


[0.47836044430732727, 0.7847769260406494]

In [37]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [38]:
# look at a single prediction
model_1_pred_probs[0]

array([0.34497398], dtype=float32)

In [39]:
# look at the first 10 predictions
model_1_pred_probs[10]

array([0.13096856], dtype=float32)

In [40]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [41]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.4776902887139,
 'f1_score': 0.7820385831619191,
 'precision': 0.7887661817696516,
 'recall': 0.7847769028871391}

In [42]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1_score': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [43]:
# Looks like our baseline is out performing our first deep learning model...
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embeddings

In [44]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [46]:
# Get the weight matrix of the embedding layer
# (these are the numerical representations of each token in out training data, which have been learned for ~5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)  # same size as vocab size and embedding dim (output dim of embedding layer)

(10000, 128)


Now, we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called [Projector](http://projector.tensorflow.org)

And TensorFlow also has an incredible guide on word embeddings themselves. 
https://www.tensorflow.org/text/guide/word_embeddings

In [47]:
# Create embedding files (we got this from TensorFlow's word embedding documentation)
# import io
# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# for index, word in enumerate(words_in_vocab):
#     if index == 0:
#         continue  # skip 0, it's padding.
#     vec = embed_weights[index]
#     out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#     out_m.write(word + "\n")
# out_v.close()
# out_m.close()

In [48]:
# Download files from Colab to upload to projector
# try:
#     from google.colab import files
#     files.download('vectors.tsv')
#     files.download('metadata.tsv')
# except Exception:
#     pass

Downloading the files above we can visualize them using [Projector](http://projector.tensorflow.org/) and clicking the "load" button on the left hand side.

> 📖**Resources:** If you'd like to know more about embeddings, check out:
* Jay Alammar's visualized word2vec [post](https://jalammar.github.io/illustrated-word2vec/)
* TensorFlow's Word Embeddings [guide](https://www.tensorflow.org/text/guide/word_embeddings)

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a precious input to aid the representation of a later input.

> 📖 **Resources:** If you want an overview of the internals of a recurrent neural network, see the following:
- [MIT's sequence modelling lecture](https://www.youtube.com/watch?v=qjrad0V0uJE)
- [Chris Olah's intro to LSTMs](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [Andrej Karpathy's the unreasonable effectiveness of recurrent neural networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:
```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/Dense) -> Output (label probability)
```

In [49]:
# Create an LSTM model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

inputs = Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = LSTM(units=64, return_sequences=True)(x)    # when you're stacking RNN cells together, you need to set return_sequences=True
# print(x.shape)
x = LSTM(units=64)(x)
# print(x.shape)
# x = Dense(64, activation="relu")(x)
# print(x.shape)
outputs = Dense(1, activation="sigmoid")(x)

model_2 = Model(inputs, outputs, name="model_2_LSTM")

In [50]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [51]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])

In [52]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220609-042931
Epoch 1/5
215/215 [==============================] - 6s 9ms/step - loss: 0.2213 - accuracy: 0.9222 - val_loss: 0.5877 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1554 - accuracy: 0.9419 - val_loss: 0.5857 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1337 - accuracy: 0.9508 - val_loss: 0.6644 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1056 - accuracy: 0.9585 - val_loss: 0.7155 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0899 - accuracy: 0.9670 - val_loss: 0.8369 - val_accuracy: 0.7703


In [53]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[0.15213218],
       [0.90351564],
       [0.999676  ],
       [0.14515418],
       [0.00212126],
       [0.99892455],
       [0.83245045],
       [0.99982435],
       [0.9996835 ],
       [0.6441633 ]], dtype=float32)

In [54]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [55]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.03412073490814,
 'f1_score': 0.7702586569524231,
 'precision': 0.7702000809485927,
 'recall': 0.7703412073490814}

In [56]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1_score': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 3: GRU

Another popular and effective RNN component is the GRU (Gated Recurrent Unit).

The GRU cell has similar features to am LSTM cell but has less parameters.

> 📖 **Resources:** 
 * [Wikipedia's Gated Recurrent Unit](https://en.wikipedia.org/wiki/Gated_recurrent_unit#:~:text=Gated%20recurrent%20units%20(GRUs)%20are,it%20lacks%20an%20output%20gate.)
 * [Towards Data Science's Guide to LSTM’s and GRU’s](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21)

In [64]:
# Build an RNN using the GRU cell
from tensorflow.keras.layers import Input, GRU, Dense, LSTM, GlobalAveragePooling1D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

inputs = Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = GRU(64)(x)
# print(x.shape)
# x = GRU(64, return_sequences=True)(x)   # if you want to stack recurrent layers on top of each other, you need return_sequences=True
# print(x.shape)
# x = LSTM(64, return_sequences=True)(x)
# print(x.shape)
# x = GRU(64)(x)
# x = Dense(64, activation="relu")(x)
# x = GlobalAveragePooling1D()(x)
outputs = Dense(1, activation="sigmoid")(x)

model_3 = Model(inputs, outputs, name="model_3_GRU")

In [65]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_3 (GRU)                 (None, 64)                37248     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [66]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])

In [67]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220609-044453
Epoch 1/5
215/215 [==============================] - 6s 14ms/step - loss: 0.1635 - accuracy: 0.9333 - val_loss: 0.8406 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0839 - accuracy: 0.9695 - val_loss: 0.8456 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0718 - accuracy: 0.9730 - val_loss: 1.0237 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0628 - accuracy: 0.9733 - val_loss: 1.1310 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0536 - accuracy: 0.9774 - val_loss: 1.1080 - val_accuracy: 0.7730


In [70]:
# Make some predictions with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.29793701e-03],
       [9.04804707e-01],
       [9.99846101e-01],
       [1.73979282e-01],
       [1.16494666e-04],
       [9.99686241e-01],
       [9.12172854e-01],
       [9.99951839e-01],
       [9.99881744e-01],
       [9.18074191e-01]], dtype=float32)

In [71]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [72]:
# Calculate model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.29658792650919,
 'f1_score': 0.7715589628566201,
 'precision': 0.7734793297631878,
 'recall': 0.7729658792650919}

In [73]:
# Looks like our baseline is out performing than our model_3_GRU...
import numpy as np

np.array(list(model_3_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])